In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,ForeignKey
from sqlalchemy import Column, Date, Integer, String,Float
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
#Read imdb names .csv file
names_csv=pd.read_csv('Raw_Data\\IMDb names.csv')

In [3]:
len(names_csv)

297705

In [4]:
names_csv.head()

,imdb_name_id,name,birth_name,height,bio,birth_details,date_of_birth,place_of_birth,death_details,date_of_death,place_of_death,reason_of_death,spouses_string,spouses,divorces,spouses_with_children,children
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987-06-22,"Los Angeles, California, USA",pneumonia,Robyn Smith (27 June 1980 - 22 June 1987) (hi...,2,0,1,2
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014-08-12,"New York City, New York, USA",stroke,Jason Robards (4 July 1961 - 10 September 196...,2,1,2,3
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934-09-28,"Paris, France",NaN,NaN,NaN,NaN,Bernard d'Ormale (16 August 1992 - present)\n...,4,3,1,1
3,nm0000004,John Belushi,John Adam Belushi,170.0,"John Belushi was born in Chicago, Illinois, US...","January 24, 1949 in Chicago, Illinois, USA",1949-01-24,"Chicago, Illinois, USA","March 5, 1982 in Hollywood, Los Angeles, Calif...",1982-03-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication,Judith Belushi-Pisano (31 December 1976 - 5 M...,1,0,0,0
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179.0,"Ernst Ingmar Bergman was born July 14, 1918, t...","July 14, 1918 in Uppsala, Uppsala län, Sweden",1918-07-14,"Uppsala, Uppsala län, Sweden","July 30, 2007 in Fårö, Gotlands län, Sweden (...",2007-07-30,"Fårö, Gotlands län, Sweden",natural causes,Ingrid Bergman (11 November 1971 - 20 May 199...,5,4,5,8


In [5]:
#Checks max lenghts of all string i.e. object columns
for c in names_csv:
    if names_csv[c].dtype=='object':
        print(f'{c} {names_csv[c].str.len().max()}')

imdb_name_id 10
name 54
birth_name 217
bio 40729.0
birth_details 184.0
date_of_birth 105.0
place_of_birth 167.0
death_details 171.0
date_of_death 94.0
place_of_death 112.0
reason_of_death 119.0
spouses_string 585.0


In [6]:
#If date_of_birth value is nan or length is >10 then defualt to NaN.
names_csv['date_of_birth'] = [ np.nan if pd.isnull(x) or len(x)>10  else x for x in names_csv['date_of_birth'] ]

In [7]:
#Verify Max length of the column is 10
names_csv['date_of_birth'].dropna().str.len().max()

10

In [8]:
#Convert dob column to datetime
# There is one dob of value '1564-04-23'.Pandas only recognizes dates >= '1677-09-21' (pd.Timestamp.min).
#errors='coerce' changes it outbound date values to NaT
names_csv['date_of_birth']=pd.to_datetime(names_csv['date_of_birth'],format='%Y-%m-%d',errors = 'coerce')

In [9]:
#Date Of death values with lenght grater than 10
[x for x in names_csv['date_of_death'].dropna() if len(x)>10]

['2017 in\xa0Québec, Canada',
 '2001 in\xa0Spain',
 '1999 in\xa0Rome, Lazio, Italy',
 '1972 in\xa0Beirut, Lebanon',
 '2006 in\xa0Chiltern, Buckinghamshire, England, UK',
 '1973 in\xa0Mexico',
 '1962 in\xa0Taunton, Somerset, UK',
 '1996 in\xa0Mexico, D.F., Mexico',
 '1965 in\xa0Georgetown, Guyana',
 '456 BC in\xa0Gela, Sicily, Italy',
 '1992 in\xa0Moscow, Russia',
 '1996 in\xa0Rome, Lazio, Italy',
 '1983 in\xa0Madrid, Spain',
 '1723 in\xa0Beja, Portugal',
 '1995 in\xa0Javea, Valencia, Spain',
 '1998 in\xa0Quezon City, Philippines \xa0(skin cancer)',
 '1992 in\xa0Istanbul, Turkey',
 '2001 in\xa0Istanbul, Turkey',
 '1994 in\xa0Moscow, Russia',
 '1997 in\xa0Philippines \xa0(cardiac arrest)',
 '1991 in\xa0Izmir, Turkey',
 '1959 in\xa0London, England, UK',
 '1975 in\xa0Belgrade, Yugoslavia',
 '1986 in\xa0Chiltern & Beaconsfield, Buckinghamshire, England, UK',
 '1994 in\xa0Greece',
 '1924 in\xa0France',
 '2009 in\xa0Lambeth, London, England, UK',
 '1998 in\xa0New York City, New York, USA',
 '

In [10]:
#If date_of_birth value is nan or length is >10 then defualt to NaN.
names_csv['date_of_death'] = [ np.nan if pd.isnull(x) or len(x)>10  else x for x in names_csv['date_of_death'] ]

In [11]:
#Verify Max length of the column is 10
names_csv['date_of_death'].dropna().str.len().max()

10

In [12]:
#Convert dob column to datetime
# There is one dob of value '1616-04-23'.Pandas only recognizes dates >= '1677-09-21' (pd.Timestamp.min).
#errors='coerce' changes it outbound date values to NaT
names_csv['date_of_death']=pd.to_datetime(names_csv['date_of_death'],format='%Y-%m-%d',errors = 'coerce')

In [13]:
pd.Timestamp.min

Timestamp('1677-09-21 00:12:43.145225')

In [14]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

In [15]:
names_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297705 entries, 0 to 297704
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   imdb_name_id           297705 non-null  object        
 1   name                   297705 non-null  object        
 2   birth_name             297705 non-null  object        
 3   height                 44681 non-null   float64       
 4   bio                    204698 non-null  object        
 5   birth_details          110612 non-null  object        
 6   date_of_birth          101740 non-null  datetime64[ns]
 7   place_of_birth         103992 non-null  object        
 8   death_details          39933 non-null   object        
 9   date_of_death          39129 non-null   datetime64[ns]
 10  place_of_death         37038 non-null   object        
 11  reason_of_death        22694 non-null   object        
 12  spouses_string         45352 non-null   obje

In [16]:
#declarative Base
Base = declarative_base()

In [17]:
#Create Engine movies_db
engine = create_engine("sqlite:///movies_db.sqlite")

In [18]:
# This creates the DB (movies_dv.sqllite) if it does not exist and connects to it
conn = engine.connect()

In [19]:
#Define class i.e table to be created in DB
class Names(Base):
    __tablename__='names'
    imdb_name_id = Column(String,primary_key=True)
    name = Column(String)
    birth_name = Column(String)
    height = Column(Float)
    bio  = Column(String)
    birth_details  = Column(String)
    date_of_birth = Column(Date)
    place_of_birth  = Column(String)
    death_details  = Column(String)
    date_of_death = Column(Date)
    place_of_death  = Column(String)
    reason_of_death  = Column(String)
    spouses_string  = Column(String)
    spouses = Column(Integer)
    divorces = Column(Integer)
    spouses_with_children = Column(Integer)
    children = Column(Integer)

In [20]:
#Creates 'names' table in DB
Base.metadata.create_all(engine)

In [21]:
#Inserts data from names_csv to names table in movies_db sqllite DB.
names_csv.to_sql('names',conn,index=False,if_exists='append')

In [22]:
conn.close()